In [137]:
import numpy as np
import pandas as pd
import plotly
import requests
import json
from plotly.graph_objs import Figure, Histogram, Layout, Scatter, Bar, Pie
plotly.tools.set_credentials_file(username='RAQH', api_key='s6lloVoHT3qmMF1dLbii')
import scipy as stats
import seaborn as sns
plotly.offline.init_notebook_mode(connected=True)

### Read "Open Table Data" scraped from "Open Table" site

In [138]:
opentable = pd.read_csv('opentable_data.csv') #import


In [139]:
opentable.head()

,restaurant,location,cuisine,price,food_rating,service_rating,ambience_rating,value_rating,num_reviews,recommendation_percentage,chef,dining_style,dress_code,noise_level
0,Gabriel Kreuther,Midtown West,Contemporary French / American,$50 and over,4.8,4.8,4.8,4.3,1269.0,91%,Gabriel Kreuther,Fine Dining,Business Casual,Quiet
1,Davio's Northern Italian Steakhouse - Manhattan,Midtown East,Steakhouse,$31 to $50,4.5,4.5,4.4,4.2,947.0,93%,Steve DiFillippo Chef/Owner,Fine Dining,Smart Casual,Moderate
2,Empellon Taqueria,West Village,Mexican,$31 to $50,4.5,4.6,4.4,4.1,1581.0,81%,Alex Stupak,Casual Dining,Smart Casual,Moderate
3,A√±ejo Tribeca,TriBeCa - Downtown,Mexican,$31 to $50,4.5,4.3,4.3,4.1,422.0,82%,Ricardo Camacho,Casual Dining,Smart Casual,Moderate
4,Yopparai,Lower East Side,Izakaya,$50 and over,4.8,4.8,4.7,4.4,310.0,95%,Junya Miura,Casual Elegant,Casual Dress,Quiet


### Drop duplicate restaurant and covert it to lower case

In [140]:
opentable = opentable.drop_duplicates(subset=['restaurant'])

opentable['restaurant'] = opentable['restaurant'].str.lower()
opentable.head()

,restaurant,location,cuisine,price,food_rating,service_rating,ambience_rating,value_rating,num_reviews,recommendation_percentage,chef,dining_style,dress_code,noise_level
0,gabriel kreuther,Midtown West,Contemporary French / American,$50 and over,4.8,4.8,4.8,4.3,1269.0,91%,Gabriel Kreuther,Fine Dining,Business Casual,Quiet
1,davio's northern italian steakhouse - manhattan,Midtown East,Steakhouse,$31 to $50,4.5,4.5,4.4,4.2,947.0,93%,Steve DiFillippo Chef/Owner,Fine Dining,Smart Casual,Moderate
2,empellon taqueria,West Village,Mexican,$31 to $50,4.5,4.6,4.4,4.1,1581.0,81%,Alex Stupak,Casual Dining,Smart Casual,Moderate
3,a√±ejo tribeca,TriBeCa - Downtown,Mexican,$31 to $50,4.5,4.3,4.3,4.1,422.0,82%,Ricardo Camacho,Casual Dining,Smart Casual,Moderate
4,yopparai,Lower East Side,Izakaya,$50 and over,4.8,4.8,4.7,4.4,310.0,95%,Junya Miura,Casual Elegant,Casual Dress,Quiet


### Second Data - Read restaurant violation data from NYC data
 * Rename "dba" column to "restaurant".
 * Convert "restaurant" column to lower case

In [141]:
from sodapy import Socrata
client = Socrata("data.cityofnewyork.us", None)
results = client.get("43nn-pn8j", limit=381633)

In [170]:
results_df_RAW = pd.DataFrame.from_records(results)
results_df_RAW.to_csv('RAWpractice_data.csv', index=True)
results_df_RAW = results_df_RAW.rename(columns = {"dba" : "restaurant"})
results_df_RAW['restaurant'] = results_df_RAW['restaurant'].str.lower()
results_df_RAW = results_df_RAW[results_df_RAW.boro=='MANHATTAN']
results_df_RAW.head()

,action,boro,building,camis,critical_flag,cuisine_description,restaurant,grade,grade_date,inspection_date,inspection_type,phone,record_date,score,street,violation_code,violation_description,zipcode
33,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Critical,Irish,dj reynolds pub and restaurant,A,2019-06-06T00:00:00.000,2019-06-06T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,8,WEST 57 STREET,06C,Food not protected from potential source of co...,10019
34,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Not Critical,Irish,dj reynolds pub and restaurant,A,2019-06-06T00:00:00.000,2019-06-06T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,8,WEST 57 STREET,10F,Non-food contact surface improperly constructe...,10019
35,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Critical,Irish,dj reynolds pub and restaurant,A,2018-05-16T00:00:00.000,2018-05-16T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,12,WEST 57 STREET,04L,Evidence of mice or live mice present in facil...,10019
36,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Not Critical,Irish,dj reynolds pub and restaurant,A,2018-05-16T00:00:00.000,2018-05-16T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,12,WEST 57 STREET,08A,Facility not vermin proof. Harborage or condit...,10019
37,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Not Critical,Irish,dj reynolds pub and restaurant,A,2018-05-16T00:00:00.000,2018-05-16T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,12,WEST 57 STREET,10F,Non-food contact surface improperly constructe...,10019


In [172]:
results_df = results_df_RAW[results_df_RAW.zipcode!='N/A']
results_df.head(2)

,action,boro,building,camis,critical_flag,cuisine_description,restaurant,grade,grade_date,inspection_date,inspection_type,phone,record_date,score,street,violation_code,violation_description,zipcode
33,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Critical,Irish,dj reynolds pub and restaurant,A,2019-06-06T00:00:00.000,2019-06-06T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,8,WEST 57 STREET,06C,Food not protected from potential source of co...,10019
34,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Not Critical,Irish,dj reynolds pub and restaurant,A,2019-06-06T00:00:00.000,2019-06-06T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,8,WEST 57 STREET,10F,Non-food contact surface improperly constructe...,10019


### Drop duplicate restaurant and load to another DF

In [174]:
results_df = results_df.drop_duplicates(subset=['zipcode','restaurant','street'])
results_df.head()

,action,boro,building,camis,critical_flag,cuisine_description,restaurant,grade,grade_date,inspection_date,inspection_type,phone,record_date,score,street,violation_code,violation_description,zipcode
33,Violations were cited in the following area(s).,MANHATTAN,351,30191841,Critical,Irish,dj reynolds pub and restaurant,A,2019-06-06T00:00:00.000,2019-06-06T00:00:00.000,Cycle Inspection / Initial Inspection,2122452912,2019-07-17T06:01:02.000,8,WEST 57 STREET,06C,Food not protected from potential source of co...,10019
80,Violations were cited in the following area(s).,MANHATTAN,1,40359480,Critical,American,1 east 66th street kitchen,A,2018-09-28T00:00:00.000,2018-09-28T00:00:00.000,Cycle Inspection / Initial Inspection,2128793900,2019-07-17T06:01:02.000,7,EAST 66 STREET,06D,"Food contact surface not properly washed, rins...",10065
164,Violations were cited in the following area(s).,MANHATTAN,730,40362264,Critical,American,p & s deli grocery,A,2019-05-13T00:00:00.000,2019-05-13T00:00:00.000,Cycle Inspection / Initial Inspection,2129323030,2019-07-17T06:01:02.000,9,COLUMBUS AVENUE,04H,"Raw, cooked or prepared food is adulterated, c...",10025
191,Violations were cited in the following area(s).,MANHATTAN,18,40362274,Critical,American,angelika film center,A,2019-01-02T00:00:00.000,2019-01-02T00:00:00.000,Cycle Inspection / Initial Inspection,2129952570,2019-07-17T06:01:02.000,10,WEST HOUSTON STREET,04L,Evidence of mice or live mice present in facil...,10012
228,Violations were cited in the following area(s).,MANHATTAN,60,40362715,Critical,Sandwiches/Salads/Mixed Buffet,the country cafe,Z,2019-06-07T00:00:00.000,2019-06-07T00:00:00.000,Cycle Inspection / Re-inspection,3474279132,2019-07-17T06:01:02.000,20,WALL STREET,02G,Cold food item held above 41Âº F (smoked fish ...,10005


### Find the count of "Critical and Not Critical Violation per restaurant"

In [180]:
results_df_Criticalraw = results_df_RAW[results_df_RAW.critical_flag=='Critical']
results_df_Critical = results_df_Criticalraw.groupby(["restaurant","zipcode"]).agg({'critical_flag':'count'})
results_df_Critical = results_df_Critical.rename(columns = {"critical_flag" : "CriticalViolationCount"})
results_df_Critical = results_df_Critical.reset_index(level=['restaurant','zipcode'])
results_df_Critical.head()

,restaurant,zipcode,CriticalViolationCount
0,#1 chinese restaurant,10033,13
1,$1 pizza,10001,4
2,& pizza,10001,6
3,& pizza,10003,6
4,'cesca,10023,9


In [181]:
results_df_NotCriticalraw = results_df_RAW[results_df_RAW.critical_flag=='Not Critical']
results_df_NotCritical = results_df_NotCriticalraw.groupby(["restaurant","zipcode"]).agg({'critical_flag':'count'})
results_df_NotCritical = results_df_NotCritical.rename(columns = {"critical_flag" : "NotCriticalViolationCount"})
results_df_NotCritical = results_df_NotCritical.reset_index(level=['restaurant','zipcode'])
results_df_NotCritical.head()


,restaurant,zipcode,NotCriticalViolationCount
0,#1 chinese restaurant,10033,9
1,& pizza,10001,4
2,& pizza,10003,5
3,'cesca,10023,2
4,'essen,10017,3


### Merge Critical and Not Critical Violation Dataframes

In [185]:
results_df_Criticalall = pd.merge(results_df_Critical,results_df_NotCritical ,
                        how='left',left_on=['restaurant','zipcode'],right_on=['restaurant','zipcode'])
results_df_Criticalall['NotCriticalViolationCount'] = results_df_Criticalall['NotCriticalViolationCount'].fillna(0)
results_df_Criticalall.head()

,restaurant,zipcode,CriticalViolationCount,NotCriticalViolationCount
0,#1 chinese restaurant,10033,13,9.0
1,$1 pizza,10001,4,0.0
2,& pizza,10001,6,4.0
3,& pizza,10003,6,5.0
4,'cesca,10023,9,2.0


### Merge Open table Data and Restaurant Violation Data (without duplicates)

In [182]:
Opentable_ResulttableMerge = pd.merge(opentable[['restaurant','cuisine','price', 'food_rating','dining_style','dress_code']],
                        results_df[['restaurant','grade','zipcode']], 
                        how='left',left_on=['restaurant'],right_on=['restaurant'])
Opentable_ResulttableMerge.head()

,restaurant,cuisine,price,food_rating,dining_style,dress_code,grade,zipcode
0,gabriel kreuther,Contemporary French / American,$50 and over,4.8,Fine Dining,Business Casual,A,10036
1,davio's northern italian steakhouse - manhattan,Steakhouse,$31 to $50,4.5,Fine Dining,Smart Casual,NaN,NaN
2,empellon taqueria,Mexican,$31 to $50,4.5,Casual Dining,Smart Casual,NaN,NaN
3,a√±ejo tribeca,Mexican,$31 to $50,4.5,Casual Dining,Smart Casual,NaN,NaN
4,yopparai,Izakaya,$50 and over,4.8,Casual Elegant,Casual Dress,A,10002


### Merge above table with critical violation count to do analysis

In [188]:
analysis_df = pd.merge(Opentable_ResulttableMerge, results_df_Criticalall,
                        how='left',left_on=['restaurant','zipcode'],right_on=['restaurant','zipcode'])
analysis_df.head()

,restaurant,cuisine,price,food_rating,dining_style,dress_code,grade,zipcode,CriticalViolationCount,NotCriticalViolationCount
0,gabriel kreuther,Contemporary French / American,$50 and over,4.8,Fine Dining,Business Casual,A,10036,2.0,4.0
1,davio's northern italian steakhouse - manhattan,Steakhouse,$31 to $50,4.5,Fine Dining,Smart Casual,NaN,NaN,NaN,NaN
2,empellon taqueria,Mexican,$31 to $50,4.5,Casual Dining,Smart Casual,NaN,NaN,NaN,NaN
3,a√±ejo tribeca,Mexican,$31 to $50,4.5,Casual Dining,Smart Casual,NaN,NaN,NaN,NaN
4,yopparai,Izakaya,$50 and over,4.8,Casual Elegant,Casual Dress,A,10002,13.0,4.0


### Drop rows which does not have data(nan) and rename column names to be lower case to load to SQL

In [190]:
analysis_df = analysis_df.dropna()
analysis_df.head()

,restaurant,cuisine,price,food_rating,dining_style,dress_code,grade,zipcode,CriticalViolationCount,NotCriticalViolationCount
0,gabriel kreuther,Contemporary French / American,$50 and over,4.8,Fine Dining,Business Casual,A,10036,2.0,4.0
4,yopparai,Izakaya,$50 and over,4.8,Casual Elegant,Casual Dress,A,10002,13.0,4.0
5,db bistro moderne,French,$31 to $50,4.5,Casual Dining,Smart Casual,A,10036,5.0,9.0
10,patsy's pizzeria,Pizzeria,$30 and under,4.2,Casual Dining,Casual Dress,A,10017,9.0,9.0
11,patsy's pizzeria,Pizzeria,$30 and under,4.2,Casual Dining,Casual Dress,A,10065,12.0,10.0


In [191]:
analysis_df_final = analysis_df.reset_index()
del analysis_df_final['index']
analysis_df_final = analysis_df_final.rename (columns = {"CriticalViolationCount" : "criticalviolationcount", 
                                                        "NotCriticalViolationCount" : "notcriticalviolationcount"})
analysis_df_final.head()

,restaurant,cuisine,price,food_rating,dining_style,dress_code,grade,zipcode,criticalviolationcount,notcriticalviolationcount
0,gabriel kreuther,Contemporary French / American,$50 and over,4.8,Fine Dining,Business Casual,A,10036,2.0,4.0
1,yopparai,Izakaya,$50 and over,4.8,Casual Elegant,Casual Dress,A,10002,13.0,4.0
2,db bistro moderne,French,$31 to $50,4.5,Casual Dining,Smart Casual,A,10036,5.0,9.0
3,patsy's pizzeria,Pizzeria,$30 and under,4.2,Casual Dining,Casual Dress,A,10017,9.0,9.0
4,patsy's pizzeria,Pizzeria,$30 and under,4.2,Casual Dining,Casual Dress,A,10065,12.0,10.0


### Connect to local database

In [192]:
from sqlalchemy import create_engine
rds_connection_string = "postgres:Password@localhost:5432/restaurant_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Use pandas to load csv converted DataFrame into database

In [193]:
analysis_df_final.to_sql(name='restaurant', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_location table

In [194]:
pd.read_sql_query('select * from restaurant fetch first 5 rows only', con=engine)

,restaurant,cuisine,price,food_rating,dining_style,dress_code,grade,zipcode,criticalviolationcount,notcriticalviolationcount
0,gabriel kreuther,Contemporary French / American,$50 and over,5,Fine Dining,Business Casual,A,10036,2,4
1,yopparai,Izakaya,$50 and over,5,Casual Elegant,Casual Dress,A,10002,13,4
2,db bistro moderne,French,$31 to $50,5,Casual Dining,Smart Casual,A,10036,5,9
3,patsy's pizzeria,Pizzeria,$30 and under,4,Casual Dining,Casual Dress,A,10017,9,9
4,patsy's pizzeria,Pizzeria,$30 and under,4,Casual Dining,Casual Dress,A,10065,12,10


### Analysis
New table can be used to
1. we can analyse the rating of the restaurant along with the number of critical/non-critical violations to understand any significant pattern. Ratings are given by people, whereas Violations are recoreded by health-inspectors. We can analyse to see if the two parameters have any mutual relationship.

2. we can further analyse the grade of a restaurant outlet with the rating of the chain to uncover any pattern associated with these two parameters.

3. In the same table, we can view the dining style for the restaurant. This data can be viewed in conjunction with 
    grade to see if a particular dining style is associated with a particular grade
    violations to see if a particular dining style is associated with a more violations - ex. pickup joints tend to have more violations as compared to fine dining.
    
4. we can repeat the same analysis as mentioned in point 3 in conjunction to cusine instead of dining style as well.
